In [6]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [37]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator
video_id = 'oWZbWzAyHAE'



def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''


tran = get_transcript(video_id)


justText = ''
textAndTimestamps = ''
for text in tran:
    textAndTimestamps += str(int(text['start'])) + '\n' + text['text'] + '\n'
    justText += text['text'] + '\n'



In [41]:
topics = [{
	"start": 936,
	"finish": 965
},{
	"start": 965,
	"finish": 993
}]

In [43]:
systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a topic. Your goal is to produce:
1. Name of the topic
2. Very very small description of the topic
3. Rewrite this topic from your own perspective, user simple words, use markdown to highlight. Be the best 1-on-1 tutor
"""

def overlaps(a1, b1, a2, b2):
    int1 = pd.Interval(a1, b1)
    int2 = pd.Interval(a2, b2)
    return int1.overlaps(int2)

def getTranscriptForTopic(topic, transcript):
    start = topic['start']
    end = topic['finish']
    text = ''
    for snip in transcript:
        if overlaps(start, end, snip['start'], snip['start'] + snip['duration']):
            text+=snip['text']+' '
    return text

def getTopic(topic, video_id):
    return {
        "name": "",
        "desctiption": "",
        "texts": ["text1.md"],
        "video_parts": [{
            "id": video_id,
            "start": topic["start"],
            "end": topic["finish"]
        }],
        "node_type": "lecture",
        "parent_id": "startup-school",
        "thumbnail": ""
    }
startNum = 17
startChar = 'b'

for topic in topics[:1]:
    t = getTranscriptForTopic(topic, tran)
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
              "role": "system",
                "content": systemsText
            },
            {
              "role": "user",
                "content": t
            }
        ],)
    print("====================================")
    print('"' + str(startNum) + startChar + '": ' + json.dumps(getTopic(topic, video_id), indent=4) + ',') 
    startChar = chr(ord(startChar) + 1)
    print(resp.choices[0].message.content)
    print("====================================")

"17b": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "oWZbWzAyHAE",
            "start": 936,
            "end": 965
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
1. Topic Name: Key Traits of Successful Startups
2. Brief Description: This topic outlines the key characteristics that successful startups typically exhibit to attain fast growth and capture a larger market share.
3. Rewrite: 

Let's talk about the special marks of top-ranking companies that make them stand out. Remember, we will be using simple words and ideas to explain this, so don't worry if you are a beginner.

* **Repeated business**: This means the company can get customers to keep coming back and spending more money. It's like having a reliable source of money.
* **Strong customer ties**: Once people start using the company's product or service, they just don't want to stop. Th